In [5]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
import networkx as nx
import music21 as msc
from funciones import *
import sys
import os
from music21 import corpus
import random 

env = msc.environment.UserSettings()
#env['musicxmlPath'] = r'C:\Program Files (x86)\Finale NotePad 2012\Finale NotePad.exe' #Path a la aplicacion Finale
#env['musicxmlPath'] = r'C:\Program Files (x86)\MuseScore 2\bin\MuseScore.exe'          #Path a la aplicacion MuseScore 
#env['musicxmlPath']='/usr/bin/mscore'
msc.environment.set('musescoreDirectPNGPath', 'C:\\Program Files (x86)\\MuseScore 2\\bin\\MuseScore.exe')
msc.environment.set('musicxmlPath', 'C:\\Program Files (x86)\\MuseScore 2\\bin\\MuseScore.exe')

#---------------------------------------------------------------------------------------------------------
#            FUNCIONES PARA ANALISIS DE MUSICA:
#---------------------------------------------------------------------------------------------------------
#Lista de funciones:
# f_xml2graph (cancion, nombre_parte=0,modelo='melodia')
# graficar (G, color_map='rainbow',layout='espiral', labels=False)
# ql_2_fig (ql)
# f_motifs_rhytmic (cancion,length,nombre_parte=0)
# f_motifs_tonal (cancion,length,nombre_parte=0)
# f_grado_dist_M (G)
# f_grado_dist_R (G)
# f_tabla (G,nombre)
# f_xml2graph_armonia (cancion, index)
# f_armon (cancion, indexes)
# graficar_armonias_undirected(G, color_map='rainbow',layout='espiral')
# graficar_armonias_directed(Armonias)
# f_dist_escalas (cancion, nombre_parte=0)
# f_full_graph(path)
# f_hierarchy(G)
# f_rewiring_directed(G)
# transitivity_motifs(G)

#-------------------

In [4]:

musicos_1=['mozart'] #son los más conocidos y que más biblio tienen 

for l, musico in enumerate(musicos_1):
    paths = corpus.getComposer(musico)
    paths=paths[0:20] #corta hasta los primeros 20 temas
    song = msc.converter.parse(paths[1])
    L=len(song.parts) #recorro todas las voces
    voces=song.parts #quiero que grafique todas las voces
    indexes=list(range(len(voces)))
    
    M=f_xml2graph(paths[1], nombre_parte='piano', modelo='melodia'); #obtengo el grafo
    
    Armonias_song,Tiempos_armonias_song,D,G=f_armon(paths[1], indexes)
        


In [ ]:
#toma un grafo 3D y cantidad de voces. Para cada voz va a hacer una random walk por #cada uno de dichos nodos
#devuelve 'voces' cantidad de listas de los nodos que fue tomando
def f_rndm_walk(G, voces):
    #creo un dict que tiene la voz como .key y la lista de notas como .value
    ls = {}
    enlaces=[]
    nodos=[]
    for i,voz in enumerate(voces):
        ls[str(voz)] = []

    #tendriamos que hacerlo empezando de la voz principal 
    #y cada vez q  toque algun nodo que tiene un enlace no dirigido abrir una rama para ese layer
    for i, layer in enumerate(G):
        enlaces.append(layer.edges()) 
        nodos.append(layer.nodes())
    
    
    for j, layer in enumerate(G):
        M = nx.adjacency_matrix(layer) #obtenemos la matriz de adyacencia
        #normalizamos
        for i in range(M.shape[1]):
            if (np.sum(M[i]) > 0):
                M[i] = M[i]/np.sum(M[i])
        restart_prob=M.min()

        #weights=[] hay que averiguar sobre como son las matrices de adj para multigraphs y layers


        for i, nodo in enumerate(nodos[j]):
            p=random.random()
        #Aca revisamos si todos los enlaces son dirigidos que siga el algoritmo. Si no que agregue otra caminata de borracho
        #para el otro layer
            if len(layer.in_edges(nodo))>0 or len(layer.out_edges(nodo))>0 and len(layer.edges(nodo))==0:
                if p < restart_prob:
                    #do restart
                    break

                else:
                    #choose next node    
                    #se podria hacer dos random walks, una con los pesos acumulados (cum_weights) y otras sin. 
                    nodo_ran=random.choices(nodos[j], weights=[], cum_weights=None, k=1)
                    ls[str(layer)].append(nodo_ran)


            if len(G.edges(nodo))>0: #si se encuentra con un no dirigido 
                if p < restart_prob:
                    #do restart
                    break
                else:
                    #toma el nodo del otro layer al que está conectado y empieza una random walk para ese layer   
                    ls[str(layer)].append(nodo)
                    for i, nodo in enumerate(layer):
                    #Aca revisamos si todos los enlaces son dirigidos que siga el algoritmo. Si no que agregue otra caminata de borracho
                    #para el otro layer
                        if len(layer.in_edges(nodo))>0 or len(layer.in_edges(nodo))>0 and len(layer.edges(nodo))==0:
                            if p < restart_prob:
                                #do restart

                            else:
                                #choose next node    
                                #se podria hacer dos random walks, una con los pesos acumulados (cum_weights) y otras sin. 
                                nodo_ran=random.choices(nodos, weights=[], cum_weights=None, k=1)
                                ls[str(voz)].append(nodo_ran)

    return(ls)
